In [1]:
from gensim.models import Word2Vec,FastText
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# WORD2VEC 
W2V_SIZE = 512 # try to tweak , was 300
W2V_WINDOW = 8 # context window
W2V_EPOCH = 32 # to optimize was 32 by default
W2V_MIN_COUNT = 8 # to optimize was 32 or 16 by default

In [3]:
model = FastText.load("fasttext.model")

In [4]:
print(model.wv['how'].shape)

(300,)


In [18]:
def sentence_vector(sentence, model):
    words = sentence.split(" ")
    
    # Add a neutral word if the number of words is even
    if len(words) % 2 == 0:
        words.append(".")  # Use the "." character as a neutral word, assume it is in the vocabulary

    # Prepare the sequence of vectors
    size=len(model.wv['how'])
    vectors = [model.wv[words[idx]].reshape(size,1) for idx in range(len(words))]

    # Convert the vectors to tensors
    #tensors = [tf.constant(vector, dtype=tf.float32) for vector in vectors]
    #tensors=np.array(tensors).reshape(len(words),size,1)

    # Use tf.linalg.matvec to perform the dot product operations
    sentence_vector = vectors[0]
    print(len(vectors))
    for i in range(1, len(vectors)):
        #print(i)
        if i%2==1:
            #print(sentence_vector.shape)
            #print(tensors[i].T.shape)
            #sentence_vector = tf.linalg.matvec(sentence_vector.reshape(size,1), tensors[i].reshape(1,size))
            #print(sentence_vector.shape)
            #print(vectors[i].T.shape)
            sentence_vector= sentence_vector@vectors[i].T
        else:
            #print(sentence_vector.shape)
            #print(tensors[i].shape)
            #sentence_vector = tf.linalg.matvec(sentence_vector, tensors[i])
            #print(sentence_vector.shape)
            #print(vectors[i].shape)
            sentence_vector= sentence_vector@vectors[i]
        
        # Normalize the sentence_vector after each matrix multiplication
        sentence_vector /= np.linalg.norm(sentence_vector)

    #print(sentence_vector.shape)
    return sentence_vector

def cosine_similarity_func(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]


In [24]:
# to do: normalize matrices to prevent vansihing
# Let's compute the sentence vectors for two sentences
sentence1="In the heart of a bustling city, amid the noise and the constant rush, stood a little antique shop, its charm a stark contrast to the modern structures that surrounded it, filled with stories from bygone eras, a haven for collectors and history enthusiasts."
sentence2="In the deep recesses of the unexplored ocean, away from the sun’s reach and human touch, lie mysteries untold and creatures unseen, where the cold pressure is intense and the silence unbroken, a world within our world, yet alien to our understanding."

#sentence1 = "In the vast expanse of a timeless place, where silence ruled the outer space ominously towering, it stood, the symbol of a spirit war between the one named Lucifer and the Morning Star, the ultimate of good."
#sentence2 = "Within the  realm of an endless void, where tranquility governs the cosmic space, it stood formidably, representing a celestial battle between the entity known as the Devil and the Bringer of Light, the epitome of righteousness."

vector1 = sentence_vector(sentence1, model)
vector2 = sentence_vector(sentence2, model)

# Calculate cosine similarity
similarity = cosine_similarity_func(vector1, vector2)
#print(similarity.shape)
print("Similarity: ", similarity)

45
43
Similarity:  -0.9999999
